In [1]:
# ================================================================
# 📦 Cell 1 — Imports
# ================================================================
import os, glob, time, json
from datetime import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
# ================================================================
# 📦 Cell 1 — Evaluation settings
# ================================================================
DRY_RUN = False             # Set True for no real trades
WINDOW = 50                 # same as training
# === PATHS (from your config) ===
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")
METRICS_FILE = os.path.join(MODEL_DIR, "eval_metrics.json")


In [3]:
# --------------------------------------
# Timeframe mapping
# --------------------------------------
TIMEFRAME = "M15"
TF_MAP = {
    "M1": mt5.TIMEFRAME_M1,
    "M5": mt5.TIMEFRAME_M5,
    "M15": mt5.TIMEFRAME_M15,
    "M30": mt5.TIMEFRAME_M30,
    "H1": mt5.TIMEFRAME_H1,
    "H4": mt5.TIMEFRAME_H4,
    "D1": mt5.TIMEFRAME_D1
}
tf = TF_MAP[TIMEFRAME.upper()]

In [11]:
DEFAULT_LOT = 0.1           # as requested
MAX_POS_PER_SYMBOL = 2

# SL/TP settings: SL computed from volatility (function below); TP = 3 * SL
DEFAULT_SL_PIPS_FALLBACK = 20
TP_MULT = 3

WINDOW = 50
BUFFER = 60
COUNT = WINDOW + BUFFER   # extra bars for safety

# Logging
LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")


In [5]:
# Cell 3
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

#def raw_from_safe(safe: str) -> str:
 #   return safe.replace("_", " ")

def safe_from_raw(raw: str) -> str:
    return make_safe_name(raw)


In [6]:
# ================================================================
# ⚙️ Cell 4 - Symbols Selection
# ================================================================
SYMBOLS = [
    # --- Volatility Indices (Standard) ---
    "Volatility 10 Index",
    "Volatility 25 Index",
    "Volatility 50 Index",
    "Volatility 75 Index",
    "Volatility 100 Index",

    # --- Volatility 1s Indices ---
    "Volatility 10 (1s) Index",
    "Volatility 25 (1s) Index",
    "Volatility 50 (1s) Index",
    "Volatility 75 (1s) Index",
    "Volatility 100 (1s) Index",

    # --- Volatility 10s Indices ---
    "Volatility 10 (10s) Index",
    "Volatility 25 (10s) Index",
    "Volatility 50 (10s) Index",
    "Volatility 75 (10s) Index",
    "Volatility 100 (10s) Index",

    # --- Jump Indices ---
    "Jump 10 Index",
    "Jump 25 Index",
    "Jump 50 Index",
    "Jump 75 Index",
    "Jump 100 Index",

    # --- Step Indices ---
    "Step Index 25",
    "Step Index 50",
    "Step Index 75",
    "Step Index 100",

    # --- Forex Reference ---
    "EURUSD"
]


In [12]:
# --------------------------------------
# Fetch bars for each symbol individually
# --------------------------------------
def fetch_symbol_bars(symbol):
    """Fetch bars safely for one symbol."""
    # Ensure MT5 knows this symbol

    mt5.initialize()
    
    info = mt5.symbol_info(symbol)
    if info is None:
        print(f"❌ Symbol not found in MT5: {symbol}")
        return None

    if not info.visible:
        mt5.symbol_select(symbol, True)

    bars = mt5.copy_rates_from_pos(symbol, tf, 0, COUNT)

    if bars is None or len(bars) < WINDOW + 2:
        print(f"❌ Not enough bars for {symbol} (got {0 if bars is None else len(bars)})")
        return None

    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    mt5.shutdown()
    return df


# --------------------------------------
# Fetch all symbols into a dictionary
# --------------------------------------
symbol_data = {}

for sym in SYMBOLS:
    print(f"📡 Fetching live data for: {sym}")
    df = fetch_symbol_bars(sym)
    if df is not None:
        symbol_data[sym] = df
        print(f"✔️ Loaded {len(df)} bars for {sym}")
    else:
        print(f"⚠️ Failed to load data for {sym}")

print(f"\n✅ Completed fetching live data for {len(symbol_data)} symbols.")


📡 Fetching live data for: Volatility 10 Index
✔️ Loaded 110 bars for Volatility 10 Index
📡 Fetching live data for: Volatility 25 Index
✔️ Loaded 110 bars for Volatility 25 Index
📡 Fetching live data for: Volatility 50 Index
✔️ Loaded 110 bars for Volatility 50 Index
📡 Fetching live data for: Volatility 75 Index
✔️ Loaded 110 bars for Volatility 75 Index
📡 Fetching live data for: Volatility 100 Index
✔️ Loaded 110 bars for Volatility 100 Index
📡 Fetching live data for: Volatility 10 (1s) Index
✔️ Loaded 110 bars for Volatility 10 (1s) Index
📡 Fetching live data for: Volatility 25 (1s) Index
✔️ Loaded 110 bars for Volatility 25 (1s) Index
📡 Fetching live data for: Volatility 50 (1s) Index
✔️ Loaded 110 bars for Volatility 50 (1s) Index
📡 Fetching live data for: Volatility 75 (1s) Index
✔️ Loaded 110 bars for Volatility 75 (1s) Index
📡 Fetching live data for: Volatility 100 (1s) Index
✔️ Loaded 110 bars for Volatility 100 (1s) Index
📡 Fetching live data for: Volatility 10 (10s) Index
❌ Sy

In [15]:
# ================================================================
# 📦 Cell 2 — Load PPO, Embeddings, Asset Map
# ================================================================

# Load PPO
model = PPO.load(MODEL_FILE)
print("✔ Loaded PPO model")

# Try to load VecNormalize safely
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE, venv=None)
        print("✔ Loaded VecNormalize")
    except:
        print("⚠ VecNormalize could not load — continuing without it.")

# Load embeddings
embeddings = np.load(EMBED_FILE, allow_pickle=True).item()
print("✔ Loaded embeddings:", list(embeddings.keys()))

# Load asset index map
asset_map = pd.read_csv(ASSET_MAP_FILE)
asset_to_idx = dict(zip(asset_map["asset"], asset_map["index"]))
print("✔ Loaded asset_to_idx")


✔ Loaded PPO model
⚠ VecNormalize could not load — continuing without it.


ValueError: can only convert an array of size 1 to a Python scalar

In [18]:
# Cell 4
# Load per-asset scalers saved as CSVs: {safe}_scaler.csv with columns mean/std
def load_scalers_from_csv(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        # ensure ordering of mean/std columns matches our feature order: o_pc,h_pc,l_pc,c_pc,v_pc
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        # ensure expected columns exist; otherwise try conversion as earlier notebooks did
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            # attempt auto-convert if original OHLCV present
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                raise ValueError(f"{p} missing expected columns and cannot convert")
        if len(df) > window:
            datasets[safe] = df
    return datasets

def load_embeddings(embed_file=EMBED_FILE, data_dir=DATA_DIR):
    if not os.path.exists(embed_file):
        print("No embeddings file found at", embed_file)
        return {}
    emb = np.load(embed_file, allow_pickle=True)
    # emb likely is ndarray shape (n_assets, embed_dim)
    # Map using asset_to_idx.csv if present
    emb_dict = {}
    if os.path.exists(ASSET_MAP_FILE):
        am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
        # am is mapping safe->idx
        for safe, idx in am.items():
            idx = int(idx)
            if idx < emb.shape[0]:
                emb_dict[safe] = np.array(emb[idx], dtype=np.float32)
    else:
        # fallback: map by order of normalized CSVs if sizes match
        csvs = sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv")))
        safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
        if len(safe_list) == emb.shape[0]:
            for i, safe in enumerate(safe_list):
                emb_dict[safe] = np.array(emb[i], dtype=np.float32)
        else:
            print("Warning: embedding count and CSV count mismatch; manual mapping required.")
    return emb_dict

# load
scalers = load_scalers_from_csv(DATA_DIR)
datasets = load_datasets(DATA_DIR, WINDOW)
embeddings = load_embeddings(EMBED_FILE, DATA_DIR)

print("Loaded assets:", len(datasets))
print("Loaded scalers:", len(scalers))
print("Loaded embeddings:", len(embeddings))


Loaded assets: 16
Loaded scalers: 16
Loaded embeddings: 17


In [19]:
# Cell: Safe VecNormalize loader using an inline dummy env (no external envs module required)
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
# parameters that must match training
WINDOW = 50                   # your window used in training
N_PRICE_FEATURES = 5          # o_pc,h_pc,l_pc,c_pc,v_pc
EMBED_DIM = next(iter(embeddings.values())).shape[0] if len(embeddings)>0 else 8
OBS_DIM = N_PRICE_FEATURES + 1 + EMBED_DIM   # matches: 5 + 1 + embed_dim

class _DummyTradingEnv(gym.Env):
    """
    Minimal dummy env with the correct obs/action spaces for VecNormalize.load().
    Used only to provide a 'venv' for VecNormalize.load(path, venv).
    """
    def __init__(self):
        super().__init__()
        # observation: shape=(WINDOW, OBS_DIM)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(WINDOW, OBS_DIM), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(3)  # hold / buy / sell

    def reset(self, seed=None, options=None):
        # return an observation of correct shape and empty info dict
        obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        return obs, {}

    def step(self, action):
        # return obs, reward, done, truncated, info in gymnasium style
        obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        reward = 0.0
        done = True   # immediate done is fine for dummy
        truncated = False
        info = {}
        return obs, reward, done, truncated, info

# Create vectorized dummy env with 1 env (can be >1)
venv = DummyVecEnv([lambda: _DummyTradingEnv()])

# Try loading VecNormalize with the dummy venv
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        # ensure it's in inference mode
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("Loaded VecNormalize from", VEC_FILE)
    except Exception as e:
        print("Warning: VecNormalize.load failed:", e)
        vecnorm = None
else:
    print("No VecNormalize file found at", VEC_FILE)



Loaded VecNormalize from models\multiasset\vec_normalize.pkl


In [20]:
# ================================================================
# 🛠 Cell 3 — Symbol name normalizer
# ================================================================

def safe_from_raw(symbol):
    """Match MT5 symbol name to training-safe name."""
    return symbol.strip()


In [21]:
# ================================================================
# 📊 Cell 5 — Build Obs
# ================================================================

n_features = 5
embed_dim = len(list(embeddings.values())[0])
obs_dim = n_features + embed_dim

def build_obs(df, symbol):
    """Convert MT5 DataFrame → RL observation."""
    if len(df) < WINDOW:
        return None
    
    window_df = df.iloc[-WINDOW:]

    features = window_df[["open", "high", "low", "close", "tick_volume"]].pct_change().fillna(0).values

    embed = embeddings[symbol]
    embed_rep = np.repeat(embed[np.newaxis, :], WINDOW, axis=0)

    obs = np.hstack([features, embed_rep]).astype(np.float32)

    if vecnorm:
        obs = vecnorm.normalize_obs(obs)

    return obs


In [22]:
# ================================================================
# 💹 Cell 6 — MT5 Safe Trading Wrapper
# ================================================================

def get_positions(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_order(symbol, direction, lot, sl=None, tp=None):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ No tick for:", symbol)
        return None

    price = tick.ask if direction == "BUY" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL

    if DRY_RUN:
        return {"retcode": 10009, "comment": "DRY_RUN"}

    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": order_type,
        "price": price,
        "sl": sl or 0,
        "tp": tp or 0,
        "magic": 234000,
        "deviation": 20,
        "type_filling": mt5.ORDER_FILLING_FOK
    }
    return mt5.order_send(req)


In [23]:
# ================================================================
# 🤖 Cell 7 — Prediction + Trading Logic
# ================================================================

def run_once_predict_and_manage():
    mt5.initialize()

    for symbol, df in symbol_data.items():
        print(f"\n--- {symbol} ---")

        obs = build_obs(df, symbol)
        if obs is None:
            print("⚠ Not enough data")
            continue

        action, _ = model.predict(obs, deterministic=True)

        signal = int(action)
        print("Signal:", signal)

        open_positions = get_positions(symbol)
        print("Open positions:", len(open_positions))

        if signal == 0:
            print("HOLD")
            continue

        direction = "BUY" if signal == 1 else "SELL"

        sl_pips = 0.1
        tp_pips = sl_pips * 3

        tick = mt5.symbol_info_tick(symbol)
        price = tick.ask if direction == "BUY" else tick.bid

        sl = price - sl_pips if direction == "BUY" else price + sl_pips
        tp = price + tp_pips if direction == "BUY" else price - tp_pips

        res = place_order(symbol, direction, 0.1, sl, tp)
        print("Order result:", res)

    mt5.shutdown()


In [24]:
# ================================================================
# ▶️ Cell 8 — Execute
# ================================================================
run_once_predict_and_manage()



--- Volatility 10 Index ---


KeyError: 'Volatility 10 Index'

In [ ]:
# ================================================================
# 📦 Cell 1 — Imports
# ================================================================
import os, glob, time, json
from datetime import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

# ================================================================
# 📦 Cell 1 — Evaluation settings
# ================================================================
DRY_RUN = False             # Set True for no real trades
WINDOW = 50                 # same as training
# === PATHS (from your config) ===
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")
METRICS_FILE = os.path.join(MODEL_DIR, "eval_metrics.json")

# --------------------------------------
# Timeframe mapping
# --------------------------------------
TIMEFRAME = "M15"
TF_MAP = {
    "M1": mt5.TIMEFRAME_M1,
    "M5": mt5.TIMEFRAME_M5,
    "M15": mt5.TIMEFRAME_M15,
    "M30": mt5.TIMEFRAME_M30,
    "H1": mt5.TIMEFRAME_H1,
    "H4": mt5.TIMEFRAME_H4,
    "D1": mt5.TIMEFRAME_D1
}
tf = TF_MAP[TIMEFRAME.upper()]

DEFAULT_LOT = 0.1           # as requested
MAX_POS_PER_SYMBOL = 2

# SL/TP settings: SL computed from volatility (function below); TP = 3 * SL
DEFAULT_SL_PIPS_FALLBACK = 20
TP_MULT = 3

WINDOW = 50
BUFFER = 60
COUNT = WINDOW + BUFFER   # extra bars for safety

# Logging
LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")

# Cell 3
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

#def raw_from_safe(safe: str) -> str:
 #   return safe.replace("_", " ")

def safe_from_raw(raw: str) -> str:
    return make_safe_name(raw)

# ================================================================
# ⚙️ Cell 4 - Symbols Selection
# ================================================================
SYMBOLS = [
    # --- Volatility Indices (Standard) ---
    "Volatility 10 Index",
    "Volatility 25 Index",
    "Volatility 50 Index",
    "Volatility 75 Index",
    "Volatility 100 Index",

    # --- Volatility 1s Indices ---
    "Volatility 10 (1s) Index",
    "Volatility 25 (1s) Index",
    "Volatility 50 (1s) Index",
    "Volatility 75 (1s) Index",
    "Volatility 100 (1s) Index",

    # --- Volatility 10s Indices ---
    "Volatility 10 (10s) Index",
    "Volatility 25 (10s) Index",
    "Volatility 50 (10s) Index",
    "Volatility 75 (10s) Index",
    "Volatility 100 (10s) Index",

    # --- Jump Indices ---
    "Jump 10 Index",
    "Jump 25 Index",
    "Jump 50 Index",
    "Jump 75 Index",
    "Jump 100 Index",

    # --- Step Indices ---
    "Step Index 25",
    "Step Index 50",
    "Step Index 75",
    "Step Index 100",

    # --- Forex Reference ---
    "EURUSD"
]

# --------------------------------------
# Fetch bars for each symbol individually
# --------------------------------------
def fetch_symbol_bars(symbol):
    """Fetch bars safely for one symbol."""
    # Ensure MT5 knows this symbol

    mt5.initialize()
    
    info = mt5.symbol_info(symbol)
    if info is None:
        print(f"❌ Symbol not found in MT5: {symbol}")
        return None

    if not info.visible:
        mt5.symbol_select(symbol, True)

    bars = mt5.copy_rates_from_pos(symbol, tf, 0, COUNT)

    if bars is None or len(bars) < WINDOW + 2:
        print(f"❌ Not enough bars for {symbol} (got {0 if bars is None else len(bars)})")
        return None

    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    mt5.shutdown()
    return df


# --------------------------------------
# Fetch all symbols into a dictionary
# --------------------------------------
symbol_data = {}

for sym in SYMBOLS:
    print(f"📡 Fetching live data for: {sym}")
    df = fetch_symbol_bars(sym)
    if df is not None:
        symbol_data[sym] = df
        print(f"✔️ Loaded {len(df)} bars for {sym}")
    else:
        print(f"⚠️ Failed to load data for {sym}")

print(f"\n✅ Completed fetching live data for {len(symbol_data)} symbols.")

# ================================================================
# 📦 Cell 2 — Load PPO, Embeddings, Asset Map
# ================================================================

# Load PPO
model = PPO.load(MODEL_FILE)
print("✔ Loaded PPO model")

# Try to load VecNormalize safely
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE, venv=None)
        print("✔ Loaded VecNormalize")
    except:
        print("⚠ VecNormalize could not load — continuing without it.")

# Load embeddings
embeddings = np.load(EMBED_FILE, allow_pickle=True).item()
print("✔ Loaded embeddings:", list(embeddings.keys()))

# Load asset index map
asset_map = pd.read_csv(ASSET_MAP_FILE)
asset_to_idx = dict(zip(asset_map["asset"], asset_map["index"]))
print("✔ Loaded asset_to_idx")

# Cell 4
# Load per-asset scalers saved as CSVs: {safe}_scaler.csv with columns mean/std
def load_scalers_from_csv(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        # ensure ordering of mean/std columns matches our feature order: o_pc,h_pc,l_pc,c_pc,v_pc
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        # ensure expected columns exist; otherwise try conversion as earlier notebooks did
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            # attempt auto-convert if original OHLCV present
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                raise ValueError(f"{p} missing expected columns and cannot convert")
        if len(df) > window:
            datasets[safe] = df
    return datasets

def load_embeddings(embed_file=EMBED_FILE, data_dir=DATA_DIR):
    if not os.path.exists(embed_file):
        print("No embeddings file found at", embed_file)
        return {}
    emb = np.load(embed_file, allow_pickle=True)
    # emb likely is ndarray shape (n_assets, embed_dim)
    # Map using asset_to_idx.csv if present
    emb_dict = {}
    if os.path.exists(ASSET_MAP_FILE):
        am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
        # am is mapping safe->idx
        for safe, idx in am.items():
            idx = int(idx)
            if idx < emb.shape[0]:
                emb_dict[safe] = np.array(emb[idx], dtype=np.float32)
    else:
        # fallback: map by order of normalized CSVs if sizes match
        csvs = sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv")))
        safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
        if len(safe_list) == emb.shape[0]:
            for i, safe in enumerate(safe_list):
                emb_dict[safe] = np.array(emb[i], dtype=np.float32)
        else:
            print("Warning: embedding count and CSV count mismatch; manual mapping required.")
    return emb_dict

# load
scalers = load_scalers_from_csv(DATA_DIR)
datasets = load_datasets(DATA_DIR, WINDOW)
embeddings = load_embeddings(EMBED_FILE, DATA_DIR)

print("Loaded assets:", len(datasets))
print("Loaded scalers:", len(scalers))
print("Loaded embeddings:", len(embeddings))

# Cell: Safe VecNormalize loader using an inline dummy env (no external envs module required)
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
# parameters that must match training
WINDOW = 50                   # your window used in training
N_PRICE_FEATURES = 5          # o_pc,h_pc,l_pc,c_pc,v_pc
EMBED_DIM = next(iter(embeddings.values())).shape[0] if len(embeddings)>0 else 8
OBS_DIM = N_PRICE_FEATURES + 1 + EMBED_DIM   # matches: 5 + 1 + embed_dim

class _DummyTradingEnv(gym.Env):
    """
    Minimal dummy env with the correct obs/action spaces for VecNormalize.load().
    Used only to provide a 'venv' for VecNormalize.load(path, venv).
    """
    def __init__(self):
        super().__init__()
        # observation: shape=(WINDOW, OBS_DIM)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(WINDOW, OBS_DIM), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(3)  # hold / buy / sell

    def reset(self, seed=None, options=None):
        # return an observation of correct shape and empty info dict
        obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        return obs, {}

    def step(self, action):
        # return obs, reward, done, truncated, info in gymnasium style
        obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        reward = 0.0
        done = True   # immediate done is fine for dummy
        truncated = False
        info = {}
        return obs, reward, done, truncated, info

# Create vectorized dummy env with 1 env (can be >1)
venv = DummyVecEnv([lambda: _DummyTradingEnv()])

# Try loading VecNormalize with the dummy venv
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        # ensure it's in inference mode
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("Loaded VecNormalize from", VEC_FILE)
    except Exception as e:
        print("Warning: VecNormalize.load failed:", e)
        vecnorm = None
else:
    print("No VecNormalize file found at", VEC_FILE)


# ================================================================
# 🛠 Cell 3 — Symbol name normalizer
# ================================================================

def safe_from_raw(symbol):
    """Match MT5 symbol name to training-safe name."""
    return symbol.strip()

# ================================================================
# 📊 Cell 5 — Build Obs
# ================================================================

n_features = 5
embed_dim = len(list(embeddings.values())[0])
obs_dim = n_features + embed_dim

def build_obs(df, symbol):
    """Convert MT5 DataFrame → RL observation."""
    if len(df) < WINDOW:
        return None
    
    window_df = df.iloc[-WINDOW:]

    features = window_df[["open", "high", "low", "close", "tick_volume"]].pct_change().fillna(0).values

    embed = embeddings[symbol]
    embed_rep = np.repeat(embed[np.newaxis, :], WINDOW, axis=0)

    obs = np.hstack([features, embed_rep]).astype(np.float32)

    if vecnorm:
        obs = vecnorm.normalize_obs(obs)

    return obs

# ================================================================
# 💹 Cell 6 — MT5 Safe Trading Wrapper
# ================================================================

def get_positions(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_order(symbol, direction, lot, sl=None, tp=None):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ No tick for:", symbol)
        return None

    price = tick.ask if direction == "BUY" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL

    if DRY_RUN:
        return {"retcode": 10009, "comment": "DRY_RUN"}

    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": order_type,
        "price": price,
        "sl": sl or 0,
        "tp": tp or 0,
        "magic": 234000,
        "deviation": 20,
        "type_filling": mt5.ORDER_FILLING_FOK
    }
    return mt5.order_send(req)

# ================================================================
# 🤖 Cell 7 — Prediction + Trading Logic
# ================================================================

def run_once_predict_and_manage():
    mt5.initialize()

    for symbol, df in symbol_data.items():
        print(f"\n--- {symbol} ---")

        obs = build_obs(df, symbol)
        if obs is None:
            print("⚠ Not enough data")
            continue

        action, _ = model.predict(obs, deterministic=True)

        signal = int(action)
        print("Signal:", signal)

        open_positions = get_positions(symbol)
        print("Open positions:", len(open_positions))

        if signal == 0:
            print("HOLD")
            continue

        direction = "BUY" if signal == 1 else "SELL"

        sl_pips = 0.1
        tp_pips = sl_pips * 3

        tick = mt5.symbol_info_tick(symbol)
        price = tick.ask if direction == "BUY" else tick.bid

        sl = price - sl_pips if direction == "BUY" else price + sl_pips
        tp = price + tp_pips if direction == "BUY" else price - tp_pips

        res = place_order(symbol, direction, 0.1, sl, tp)
        print("Order result:", res)

    mt5.shutdown()

# ================================================================
# ▶️ Cell 8 — Execute
# ================================================================
run_once_predict_and_manage()
